In [1]:
from geebap import scores

In [2]:
import ee

In [3]:
from geetools import tools, ui, cloud_mask, algorithms

In [4]:
Map = ui.Map()

In [5]:
Map.show()

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

In [6]:
p = ee.Geometry.Point([-71.33972167968751, -42.737619925503054])
aoi = p.buffer(8000).bounds()

Masked image

In [7]:
i = ee.Image('COPERNICUS/S2/20181122T142749_20181122T143353_T18GYT')

In [8]:
masked = cloud_mask.sentinel2()(i)
masked = masked.clip(aoi)

In [9]:
vis = {'bands':['B8', 'B11', 'B4'], 'min':0, 'max':5000}

In [10]:
Map.addLayer(masked, vis, 'Masked')

In [11]:
Map.centerObject(masked)

## CloudDist score

In [12]:
cld = scores.CloudDist()

In [13]:
help(cld.compute)

Help on function compute in module geebap.scores:

compute(img, **kwargs)
    Compute Cloud Distance score.
    
    :param kernel: a kernel
    :type kernel: ee.Kernel
    :param dmax: Maximum distance to calculate the score. If the pixel is
        further than dmax, the score will be 1.
    :type dmax: int
    :param dmin: Minimum distance.
    :type dmin: int
    :param bandmask: the band that contains the mask. Defaults to the first
        band of the image
    :type bandmask: str
    :param bandname: the name of the resulting band. Defaults to
        'cloud_dist'
    :type bandname: str
    :param units: units for the kernel. Can be 'pixels' or 'meters'. Defaults to the latter
    :type units: str



In [14]:
cldband = cld.compute(masked, bandmask='B2', dmin=0, dmax=600)

In [15]:
Map.addLayer(cldband, {'min':0, 'max':1, 'palette':['white', 'red', 'orange', 'yellow', 'green', 'blue', 'black']}, 'Cloud Distance Score')

## Using BAP's collection module

In [16]:
from geebap import satcol

In [17]:
col = satcol.Collection.Sentinel2()

In [18]:
icol = ee.ImageCollection.fromImages([masked])

In [19]:
newcol = cld.map(icol, col=col)

In [20]:
newi = ee.Image(newcol.first())

In [21]:
Map.addLayer(newi, {'bands':[cld.name], 'min':0, 'max':1}, 'Cloud Distance Score from satcol')